In [ ]:
import cv2
import pafy
print(cv2.__version__)
import av
from pydub import AudioSegment
import uuid
import boto3
import json


In [ ]:
video_url = "https://www.youtube.com/watch?v=pgYmY6--DjI"
filename = "sharks_leaping_out_of_water"
dirname = 'sharks/'
base_path = 'data/'

In [ ]:
# video_url = "https://www.youtube.com/watch?v=yqzVI0CSKCU"
# For now select the first streams, might want to go with the lowest quality streams
video = pafy.new(video_url)
video_stream = video.videostreams[0]
audio_stream = video.audiostreams[0]
video.title, video_stream.quality, audio_stream.quality

In [ ]:
# Download the audio and video files (hardcoded as webm)
video_filetype = '.webm'
audio_filetype = '.webm'
video_file_path = video_stream.download(filepath=base_path + 'video/' + dirname + filename + video_filetype)
audio_file_path = audio_stream.download(filepath=base_path + 'audio/' + dirname + filename + audio_filetype)

In [ ]:
# Get Frames from video file

vidcap = cv2.VideoCapture(video_file_path)
success,image = vidcap.read()
count = 0
success = True
frame_rate = 30

while success:
    
    success, image = vidcap.read()
    count += 1
    
    if count % frame_rate * 2 == 0:
        cv2.imwrite(f"data/frames/{dirname}{filename}_frame_{count}.jpg", image)     # save frame as JPEG file



In [ ]:
# Export audio file to local wav file
# Convert file format to an acceptable aws Transcribe file format
audio = AudioSegment.from_file(audio_file_path)
audio_response = audio.export(f'data/audio/{dirname}{filename}_audio.wav', format='wav')
audio_response.close()

In [ ]:
# Play audio as a check
audio

In [ ]:
# Get AWS S3 Bucket resources
s3_resource = boto3.resource('s3')
audio_bucket = s3_resource.Bucket('zebra-audio')
text_bucket = s3_resource.Bucket('zebra-text')

In [ ]:
# Upload audio file to S3
audio_bucket.upload_file(Filename=f'data/audio/{dirname}{filename}_audio.wav', Key=f"{filename}_audio.wav")

In [ ]:
# Create an aws transcribe client in the same region as the s3 buckets
transcribe_client = boto3.client('transcribe', region_name='us-west-2')

In [ ]:
# Start a Transcription job
response = transcribe_client.start_transcription_job(
    TranscriptionJobName=f"{filename}_transcript",
    LanguageCode='en-US',
    MediaFormat='wav',
    Media={
        'MediaFileUri': f"https://s3-us-west-2.amazonaws.com/zebra-audio/{filename}_audio.wav",
    },
    OutputBucketName=f'zebra-text'
)

In [ ]:
# Get body of transcribed file
bucket_objs = text_bucket.objects.all()
for ob in bucket_objs:
    if f"{filename}_transcript" in ob.key:
        body = ob.get()['Body'].read()

In [ ]:
# Load transcript body into json and print raw text
try:
    text_json = json.loads(body)
    transcript = text_json['results']['transcripts'][0]['transcript']
    print(transcript)
except NameError:
    print(f"Transcription Job is not done, try again in a few minutes")